In [ ]:
!pip install accelerate peft bitsandbytes transformers trl


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.0/225.0 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.1/80.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━

In [ ]:
# from huggingface_hub import notebook_login
# notebook_login()

In [ ]:
# load the required packages.

import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer
import os
from sklearn.model_selection import train_test_split

In [ ]:
#dataset="burkelibbey/colors"
model_id="TinyLlama/TinyLlama-1.1B-Chat-v1.0"
output_model="/content/drive/MyDrive/model_save/"

In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict
n=8000
csv_path = "/content/drive/MyDrive/training.json"
df = pd.read_json(csv_path)
df = df.sample(n)
train,test = train_test_split(df, test_size=0.2, random_state=42)
dataset_tr =Dataset.from_pandas(train)
dataset_te =Dataset.from_pandas(test)
dataset = DatasetDict({"train": dataset_tr, "test": dataset_te})

print(dataset)
print(dataset['train'][0])

DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion', '__index_level_0__'],
        num_rows: 6400
    })
    test: Dataset({
        features: ['prompt', 'completion', '__index_level_0__'],
        num_rows: 1600
    })
})
{'prompt': 'Read the following references:\nReference [1]: a scout. The company had always planned to set up shop at the new airfield in Speyer, but they initially had problems securing land for a factory. The Gustav designs were actually built in the Speyer Festival Hall, which was unused at the time. It was not until February 6, 1914, that the city agreed to sell Pfalz 7,000 m² to build their factory. Construction was completed in July, only one month before the start of World War I. By this point, the company had arranged a license to produce Morane-Saulnier monoplanes, which were quickly put into German service. When\nReference [2]: Pfalz Flugzeugwerke Pfalz Flugzeugwerke was a World War I German aircraft manufacturer, located at the Spey

In [ ]:
import pandas as pd

def replace_text(examples):
    # Replace 'Text' with '#text' and 'Read the following references' with '#references'
    examples['prompt'] = examples['prompt'].replace('Text:', '#text:') \
                                           .replace('Read the following references:', '#references:') \
                                           .replace("Please identify all the errors in the following passage using the references provided and suggest edits:", "")
    return examples

# Apply the function to both the train and test datasets
dataset = dataset.map(replace_text, batched=False, num_proc=1)

# Print the modified first entry of the train dataset to verify changes
print(dataset['train'][1])

Map:   0%|          | 0/6400 [00:00<?, ? examples/s]

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

{'prompt': '#references:\nReference [1]: Ltd. In 1883, after the death of his father William Froude, Richard inherited the rights to his father\'s patents, including that for the manufacture of the water brake dynamometer. From its base, the company undertook a number of significant late-Victorian era engineering projects, including supplying and constructing the steelworks for Folkestone Pier (1887) and in the same year supplying a girder bridge to Empresa de los Ferrocarriles del Estado (EFE) the national railway of Chile. The company developed two designs of steerable torpedo, the first to the design of Colonel Lay which were demonstrated to both the Admiralty, and the\nReference [2]: Heenan & Froude was a United Kingdom-based engineering company, founded in Newton Heath, Manchester, England in 1881 in a partnership formed by engineers Richard Froude and Richard Hammersley Heenan.<ref name=Slater/><ref name=Karen /><ref name=1820settlers.com /> Expanded on the back of William Froude

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataset_train = dataset['train']
dataset_test = dataset['test']


In [ ]:
dataset_train[0]

{'prompt': '#references:\nReference [1]: a scout. The company had always planned to set up shop at the new airfield in Speyer, but they initially had problems securing land for a factory. The Gustav designs were actually built in the Speyer Festival Hall, which was unused at the time. It was not until February 6, 1914, that the city agreed to sell Pfalz 7,000 m² to build their factory. Construction was completed in July, only one month before the start of World War I. By this point, the company had arranged a license to produce Morane-Saulnier monoplanes, which were quickly put into German service. When\nReference [2]: Pfalz Flugzeugwerke Pfalz Flugzeugwerke was a World War I German aircraft manufacturer, located at the Speyer airfield in the Palatinate (German: Pfalz). They are best known for their series of fighters, notably the Pfalz D.III and Pfalz D.XII. The company went bankrupt after the Armistice, when the French occupation forces confiscated all of the equipment, but the facto

### Data preparation

In [ ]:
dataset_test[0]

{'prompt': '#references:\nReference [1]: are also various research stations doing field ecological research such as the Tiputini Biodiversity Station in the Ecuadorian Amazon, Comoé National Park Research Station in the Savannas of North-eastern Côte d\'Ivoire or the Gombe Research Station in Tanzania, where famous chimpanzee research was conducted by Jane Goodall. Research station A research station is a station that is built for the purpose of conducting scientific research. Research station sites might include remote areas of the world, oceans, as well as outer space, such as the International Space Station. Some research stations are located in the Arctic, such as the Northeast Science\nReference [2]: planning and training room, and a water-treatment room. In order to minimize any effect that the main station\'s regular operations might have on the accuracy of scientific projects, small remote platforms are set up at a distance of from the main station. Magnetic, seismic, trace ele

In [ ]:
def formatted_train(system, input_text, response) -> str:
    """
    Formats the training data in ChatML format.

    Parameters:
    - system: The system prompt or task description.
    - input_text: The input text or prompt.
    - response: The response or completion.

    Returns:
    A string formatted in ChatML format.
    """
    #return f"user\n{input_text}\nassistant\n{response}\n"
    return f"<|im_start|>system{system}<|im_end|>\n<|im_start|>user\n{input_text}<|im_end|>\n<|im_start|>assistant\n{response}<|im_end|>\n"

In [ ]:
#system_prompt--> You are a halluciantion detector
#user --> Read the following references:....., Text
#response --> Completion

In [ ]:
print(formatted_train("You are a halluciantion detector, please identify all the errors in the #text using the #references provided and suggest edits.", dataset_train[0]["prompt"], dataset_train[0]["completion"]))

<|im_start|>systemYou are a halluciantion detector, please identify all the errors in the #text using the #references provided and suggest edits.<|im_end|>
<|im_start|>user
#references:
Reference [1]: a scout. The company had always planned to set up shop at the new airfield in Speyer, but they initially had problems securing land for a factory. The Gustav designs were actually built in the Speyer Festival Hall, which was unused at the time. It was not until February 6, 1914, that the city agreed to sell Pfalz 7,000 m² to build their factory. Construction was completed in July, only one month before the start of World War I. By this point, the company had arranged a license to produce Morane-Saulnier monoplanes, which were quickly put into German service. When
Reference [2]: Pfalz Flugzeugwerke Pfalz Flugzeugwerke was a World War I German aircraft manufacturer, located at the Speyer airfield in the Palatinate (German: Pfalz). They are best known for their series of fighters, notably th

In [ ]:
system_prompt = "You are a halluciantion detector, please identify all the errors in the #text using the #references provided and suggest edits."

In [ ]:
from datasets import Dataset

def prepare_train_data(dataset, system_prompt):
    """
    Prepares the training data by applying formatting to each record in the dataset.

    Parameters:
    - dataset: The dataset object containing 'prompt' and 'completion' fields.
    - system_prompt: A constant string representing the system's task description.

    Returns:
    A dataset with each record formatted in ChatML format.
    """
    # Define a helper function to apply formatting
    def format_row(row):
        # Assuming 'prompt' is your input_text and 'completion' is your response
        return formatted_train(system=system_prompt, input_text=row['prompt'], response=row['completion'])

    # Convert the dataset to a pandas DataFrame if it's not already one
    if isinstance(dataset, Dataset):
        data_df = dataset.to_pandas()
    else:
        data_df = dataset

    # Apply formatting to each record using the 'format_row' helper function
    data_df["chatml_formatted_text"] = data_df.apply(format_row, axis=1)

    # Drop the original 'prompt' and 'completion' columns if no longer needed
    # data_df = data_df.drop(['prompt', 'completion'], axis=1)

    # Convert the DataFrame back to a Hugging Face dataset
    formatted_dataset = Dataset.from_pandas(data_df)

    return formatted_dataset


In [ ]:
system_prompt = "You are a halluciantion detector, please identify all the errors in the #text using the #references provided and suggest edits."

formatted_dataset = prepare_train_data(dataset_train, system_prompt)

# Now 'formatted_dataset' contains the data formatted in ChatML format
# To verify, you can print out the first example
print(formatted_dataset[0]['chatml_formatted_text'])

<|im_start|>systemYou are a halluciantion detector, please identify all the errors in the #text using the #references provided and suggest edits.<|im_end|>
<|im_start|>user
#references:
Reference [1]: a scout. The company had always planned to set up shop at the new airfield in Speyer, but they initially had problems securing land for a factory. The Gustav designs were actually built in the Speyer Festival Hall, which was unused at the time. It was not until February 6, 1914, that the city agreed to sell Pfalz 7,000 m² to build their factory. Construction was completed in July, only one month before the start of World War I. By this point, the company had arranged a license to produce Morane-Saulnier monoplanes, which were quickly put into German service. When
Reference [2]: Pfalz Flugzeugwerke Pfalz Flugzeugwerke was a World War I German aircraft manufacturer, located at the Speyer airfield in the Palatinate (German: Pfalz). They are best known for their series of fighters, notably th

In [ ]:
formatted_dataset.shape


(6400, 4)

In [ ]:
formatted_dataset[0]

{'prompt': '#references:\nReference [1]: a scout. The company had always planned to set up shop at the new airfield in Speyer, but they initially had problems securing land for a factory. The Gustav designs were actually built in the Speyer Festival Hall, which was unused at the time. It was not until February 6, 1914, that the city agreed to sell Pfalz 7,000 m² to build their factory. Construction was completed in July, only one month before the start of World War I. By this point, the company had arranged a license to produce Morane-Saulnier monoplanes, which were quickly put into German service. When\nReference [2]: Pfalz Flugzeugwerke Pfalz Flugzeugwerke was a World War I German aircraft manufacturer, located at the Speyer airfield in the Palatinate (German: Pfalz). They are best known for their series of fighters, notably the Pfalz D.III and Pfalz D.XII. The company went bankrupt after the Armistice, when the French occupation forces confiscated all of the equipment, but the facto

In [ ]:
## The same for test data

formatted_dataset_test = prepare_train_data(dataset_test, system_prompt)

# Now 'formatted_dataset' contains the data formatted in ChatML format
# To verify, you can print out the first example
print(formatted_dataset_test[0]['chatml_formatted_text'])
formatted_dataset_test.shape

<|im_start|>systemYou are a halluciantion detector, please identify all the errors in the #text using the #references provided and suggest edits.<|im_end|>
<|im_start|>user
#references:
Reference [1]: Municipal Courts. The Supreme Court, the commonwealth's highest court, holds its sessions in San Juan's Miramar district. Puerto Rico is divided into 78 divisions, each municipality headed by a mayor. The municipalities also have a municipal legislature, which is in charge of overseeing the mayor’s operations, holding public meetings, and enacting municipal resolutions and ordinances. Both the mayor and the municipal legislators are elected at-large by the municipality’s citizens in general elections held every four years. Unlike most towns and cities in the United States., Puerto Rico does not have local or state sheriffs, sheriff duties are instead performed by
Reference [2]: The municipalities of Puerto Rico (Spanish: "municipios de Puerto Rico") are the second-level administrative div

(1600, 4)

### Model the Model (not the base version)

In [ ]:
def get_model_and_tokenizer(model_id):
    # Load the tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.pad_token = tokenizer.eos_token  # Ensure this is intended for your model's functionality

    # Configuration for BitsAndBytes quantization
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype="float16",
        bnb_4bit_use_double_quant=True
    )

    # Load the model with the specified quantization configuration
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb_config,
        device_map="auto"  # Automatically assigns the model's layers to available devices
    )

    # Adjust model configurations if necessary for your tasks
    model.config.use_cache = False  # Consider the impact on generative tasks
    model.config.pretraining_tp = 1  # Ensure this setting suits your model's architecture

    return model, tokenizer

In [ ]:
# !pip install -i https://test.pypi.org/simple/bitsandbytes
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments


In [ ]:
model, tokenizer = get_model_and_tokenizer(model_id)

In [ ]:
! nvidia-smi

NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

### Setting up the LoRA

In [ ]:
peft_config = LoraConfig(
        r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM"
    )

In [ ]:
training_arguments = TrainingArguments(
        output_dir=output_model,
        per_device_train_batch_size=8,
        gradient_accumulation_steps=4,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        logging_steps=10,
        num_train_epochs=10,
        fp16=True,
        save_strategy="epoch",
        evaluation_strategy="epoch",
        logging_strategy="epoch",
        load_best_model_at_end=True
        # push_to_hub=True,

    )

In [ ]:
?TrainingArguments

In [ ]:
from datasets import Dataset

# Map the formatted_dataset to extract only the chatml_formatted_text for each example
formatted_dataset = formatted_dataset.map(lambda examples: {'text': examples['chatml_formatted_text']}, batched=True)

NameError: name 'formatted_dataset' is not defined

In [ ]:
formatted_dataset_test = formatted_dataset_test.map(lambda examples: {'text': examples['chatml_formatted_text']}, batched=True)


Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

In [ ]:
trainer = SFTTrainer(
        model=model,
        train_dataset=train_dataset,
        peft_config=peft_config,
        dataset_text_field="text",
        args=training_arguments,
        tokenizer=tokenizer,
        packing=False,
        max_seq_length=1024,
        eval_dataset=formatted_dataset_test
    )

NameError: name 'train_dataset' is not defined

In [ ]:
! nvidia-smi

Sat Apr  6 20:02:09 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0              55W / 400W |  32053MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
0,1.488100,1.405946
1,1.403100,1.389082
2,1.388400,1.381255
4,1.373400,1.374567
5,1.368700,1.373062
6,1.365500,1.372236
8,1.362000,1.371673
9,1.358100,1.371666


Checkpoint destination directory /content/drive/MyDrive/model_save/checkpoint-156 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory /content/drive/MyDrive/model_save/checkpoint-312 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory /content/drive/MyDrive/model_save/checkpoint-468 already exists and is non-empty. Saving will proceed but saved results may be invalid.


TrainOutput(global_step=1560, training_loss=1.383287605872521, metrics={'train_runtime': 4230.0386, 'train_samples_per_second': 11.82, 'train_steps_per_second': 0.369, 'total_flos': 3.176391972696883e+17, 'train_loss': 1.383287605872521, 'epoch': 9.98})

In [ ]:
# Save the model in Google drive

trainer.model.save_pretrained("/content/drive/MyDrive/model_save")

NameError: name 'trainer' is not defined

In [ ]:
#load the. model

model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/model_save/")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

### Merging the LoRA with the base model

In [ ]:
from peft import AutoPeftModelForCausalLM, PeftModel
from transformers import AutoModelForCausalLM
import torch
import os

model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, load_in_8bit=False,
                                             device_map="auto",
                                             trust_remote_code=True)

model_path = "/content/drive/MyDrive/model_save/checkpoint-250/"

peft_model = PeftModel.from_pretrained(model, model_path, from_transformers=True, device_map="auto")

model = peft_model.merge_and_unload()

In [ ]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Line

In [ ]:
# Saving the model in Google drive

model.save_pretrained("/content/drive/MyDrive/model_save")

In [ ]:
# Load the model from Google drive

model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/model_save")


*italicized text*### Inference from the LLM

In [ ]:
from transformers import GenerationConfig
from time import perf_counter

def generate_response(user_input):

  prompt = formatted_prompt(user_input)

  inputs = tokenizer([prompt], return_tensors="pt")
  generation_config = GenerationConfig(penalty_alpha=0.6,do_sample = True,
      top_k=5,temperature=0.5,repetition_penalty=1.2,
      max_new_tokens=12,pad_token_id=tokenizer.eos_token_id
  )
  start_time = perf_counter()

  inputs = tokenizer(prompt, return_tensors="pt").to('cuda')

  outputs = model.generate(**inputs, generation_config=generation_config)
  print(tokenizer.decode(outputs[0], skip_special_tokens=True))
  output_time = perf_counter() - start_time
  print(f"Time taken for inference: {round(output_time,2)} seconds")

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import LlamaTokenizer, LlamaForCausalLM
from tqdm.auto import tqdm


model_id= "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = LlamaForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")


def generate_response(user_input):
    # Format the prompt as per your training format
    prompt = f"user\n{user_input}\nassistant:"

    # Prepare the inputs for the model
    inputs = tokenizer(prompt, return_tensors="pt").to('cuda')  # Ensure to move inputs to the same device as your model

    # Generation settings
    generation_settings = {
        "max_length": 2048,  # Adjust based on your needs
        "penalty_alpha": 0.6,
        "do_sample": True,
        "top_k": 5,
        "temperature": 0.2,
        "repetition_penalty": 1.2,
        "max_new_tokens": 50,  # Adjust the maximum tokens to generate
        "pad_token_id": tokenizer.eos_token_id,
    }

    # Generate response
    start_time = perf_counter()
    outputs = model.generate(**inputs, **generation_settings)
    output_time = perf_counter() - start_time

    # Decode and print the response
    print(f"Response: {tokenizer.decode(outputs[0], skip_special_tokens=True)}")
    print(f"Time taken for inference: {round(output_time, 2)} seconds")


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [ ]:
user_input = """
Reference: Lionel Messi is a Argentina soccer player.
Text:Lionel Messi is a England soccer player.
"""
generate_response(user_input=user_input)

Both `max_new_tokens` (=50) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Response: user

Reference: Lionel Messi is a Argentina soccer player.
Text:Lionel Messi is a England soccer player.

assistant:Can you please change the pronoun "he" to "they"?
Time taken for inference: 0.51 seconds


In [ ]:
def formatted_prompt(question)-> str:
    return f"<|im_start|>user\n{question}<|im_end|>\n<|im_start|>assistant:"

In [ ]:
def print_color_space(hex_color):
    def hex_to_rgb(hex_color):
        hex_color = hex_color.lstrip('#')
        return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))
    r, g, b = hex_to_rgb(hex_color)
    print(f'{hex_color}: \033[48;2;{r};{g};{b}m           \033[0m')

In [ ]:
generate_response(user_input='Light Orange color')

user
Light Orange color 
 assistant: #f0901a 
 light
Time taken for inference: 0.35 seconds


In [ ]:
print_color_space('#f0901a ')


#f0901a :            
